<a href="https://colab.research.google.com/github/pallavmarch/Harry-Potter-LLM-Chatbot/blob/main/HarryPotter_LLM_update2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q accelerate==0.21.0 peft==0.4.0 bitsandbytes==0.40.2 transformers==4.28.1 trl==0.4.7 pypdf langchain
!pip install --upgrade trl langchain langchain-community gradio -qq

import os
import torch
from langchain.document_loaders import DirectoryLoader, PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, TrainingArguments, pipeline
from trl import SFTTrainer
from datasets import Dataset
from peft import LoraConfig
import locale
import gradio as gr


locale.getpreferredencoding = lambda: "UTF-8"


from google.colab import userdata
sec_key = userdata.get("HF_tkn")
os.environ["HF_tkn"] = sec_key

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.0/110.0 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.5/92.5 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 61.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.4/77.4 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 kB 21.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 72.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 29.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1

In [ ]:
model_name = "NousResearch/Llama-2-13b-chat-hf"
pdf_path = "/content/J. K. Rowling - Harry Potter 1 - Sorcerer's Stone (4 files merged).pdf"
output_dir = "./results"
num_train_epochs = 3
per_device_train_batch_size = 2
per_device_eval_batch_size = 2  #4 Reduced
gradient_accumulation_steps = 2
learning_rate = 2e-4
max_seq_length = 128
device_map = "auto"

lora_r = 32
lora_alpha = 16
lora_dropout = 0.1

use_4bit = True
bnb_4bit_compute_dtype = "float16"
bnb_4bit_quant_type = "nf4"
use_nested_quant = False

loader = PyPDFLoader(pdf_path)
documents = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1200, chunk_overlap=0)
split_documents = text_splitter.split_documents(documents)
split_documents = split_documents[:int(len(split_documents) * 0.9)]    #Select the first 90% of split_documents

dataset = Dataset.from_dict({"text": [doc.page_content for doc in split_documents]})


compute_dtype = getattr(torch, bnb_4bit_compute_dtype)
bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)


model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map=device_map
)
model.config.use_cache = False



tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/608 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/33.4k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/9.95G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/9.90G [00:00<?, ?B/s]

...

You are calling `save_pretrained` to a 4-bit converted model, but your `bitsandbytes` version doesn't support it. If you want to save 4-bit models, make sure to have `bitsandbytes>=0.41.3` installed.


tokenizer_config.json:   0%|          | 0.00/746 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/21.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/435 [00:00<?, ?B/s]

In [ ]:
peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM"
)


training_arguments = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    learning_rate=learning_rate,
    fp16=True,
    logging_dir="./logs",
    logging_steps=25
)


In [ ]:
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
)

trainer.train()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': dataset_text_field, max_seq_length. Will not be supported from version '0.13.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:300: UserWarning: You passed a `max_seq_length` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:328: UserWarning: You passed a `dataset_text_field` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(


Map:   0%|          | 0/2554 [00:00<?, ? examples/s]

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Step,Training Loss
25,2.308800
50,2.048700
75,1.990200
100,1.958700
125,1.928800
150,1.925000
175,1.887200
200,1.892200
225,1.924000
250,1.882900


Step,Training Loss
25,2.308800
50,2.048700
75,1.990200
100,1.958700
125,1.928800
150,1.925000
175,1.887200
200,1.892200
225,1.924000
250,1.882900


TrainOutput(global_step=1914, training_loss=1.7696992102835245, metrics={'train_runtime': 5783.2214, 'train_samples_per_second': 1.325, 'train_steps_per_second': 0.331, 'total_flos': 7.47094960272384e+16, 'train_loss': 1.7696992102835245, 'epoch': 2.9976507439310884})

In [ ]:
def generate_text(prompt, model, tokenizer, max_length=250):
    instruction = f"[INST] {prompt} [/INST]"
    pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=max_length,do_sample=True, temperature=0.7)
    result = pipe(instruction)
    return result[0]['generated_text']

In [ ]:
prompt = "What is the name of Hagrid's dog?"
generated_text = generate_text(prompt, model, tokenizer)
print(generated_text)


[INST] What is the name of Hagrid's dog? [/INST]  Hagrid's dog is named Fang.


In [ ]:
prompt = "What are horcrux?"
generated_text = generate_text(prompt, model, tokenizer)
print(generated_text)

[INST] What are horcrux? [/INST]  Horcruxes are a central concept in the Harry Potter series by J.K. Rowling. They are objects that contain a piece of a wizard's soul, which is created by murdering someone. The idea of horcruxes is that a wizard can split his or her soul into two parts, one of which remains in the body, and the other of which is placed in an object, creating a kind of soul-split. 
 
The idea of horcruxes is that a wizard can split his or her soul into two parts, one of which remains in the body, and the other of which is placed in an object, creating a kind of soul-split. The soul piece in the object is called a "horcrux." 
 
The main villain of the series, Lord Voldemort, created seven horcruxes to ensure his immortality. He created the first horcrux when he tried to kill Harry Potter as a baby, and then created six more by murdering people he considered to be important to him. 
 
The


In [ ]:
prompt = "Is Malfoy an ally of Voldemort?"
generated_text = generate_text(prompt, model, tokenizer)
print(generated_text)


[INST] Is Malfoy an ally of Voldemort? [/INST]  In the Harry Potter series, Draco Malfoy is not an ally of Lord Voldemort, but he does have some connections to the Dark Lord and his followers. Here are some key points about Malfoy's relationship with Voldemort: 
 
1. Malfoy's family is one of the most well-known pure-blood wizarding families, and they have a long history of prejudice against Muggle-born wizards and those with Muggle relatives. This prejudice aligns with Voldemort's belief in the superiority of pure-blood wizards. 
 
2. Malfoy is a student at Hogwarts during Voldemort's rise to power, and he is a member of the Slytherin house, which is known for producing many of the Dark Lord's followers. 
 
3. Malfoy is a member of the Hogwarts Quidditch team, and he becomes friends with Crabbe and Goyle, two other S


In [ ]:
prompt = "Give me 5 examples of cool potions and explain what they do?"
generated_text = generate_text(prompt, model, tokenizer)
print(generated_text)


[INST] Give me 5 examples of cool potions and explain what they do? [/INST]  Sure, here are five examples of cool potions and what they do: 
 
1. Draught of Living Death - This potion will cause the drinker to become 
completely lifeless and immobile. It is often used by wizards who want 
to take a long, safe nap in a crowded place, or by those who want to 
avoid danger. 
 
2. Elixir of Love - This potion will cause the drinker to fall in love 
with the first living creature they set eyes on. It is often used by 
those who want to get over a broken heart, or by those who want to 
fall in love with someone they can't have. 
 
3. Felix Felicis - This potion will make the drinker extremely lucky for 
a short period of time. It is often used by students before exams, or 
by those who want to get out of a difficult situation. 
 
4. Polyjuice Potion


In [ ]:
prompt = "Explain me the significance of the Triwizard Tournament in Harry Potter."
generated_text = generate_text(prompt, model, tokenizer)
print(generated_text)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


[INST] Explain me the significance of the Triwizard Tournament in Harry Potter. [/INST]  The Triwizard Tournament is a magical competition that takes place every five years between three schools of magic: Hogwarts, Beauxbatons, and Durmstrang. The tournament is significant in the Harry Potter series for several reasons: 
 
1. Dangerous tasks: The Triwizard Tournament involves three champions, one from each school, completing a series of dangerous tasks. These tasks are designed to test the champions' magical prowess, bravery, and cunning, and are often life-threatening. 
 
2. Age limit: The tournament is open only to students over the age of seventeen, which makes Harry Potter ineligible to participate when he is only sixteen. This age limit is significant because it forces Harry to watch the tournament from the sidelines, unable to participate, and because it highlights the dangers of the tournament. 
 
3. Unfair advantages: The tournament is also significant because it reveals the in

In [ ]:
prompt = "Which challenges does Harry face during the Triwizard Tournament"
generated_text = generate_text(prompt, model, tokenizer)
print(generated_text)

[INST] Which challenges does Harry face during the Triwizard Tournament [/INST]  During the Triwizard Tournament, Harry faces several challenges, including:

1. The First Task: The first task is to retrieve a golden egg from underwater, guarded by merpeople. Harry must retrieve the egg before the other champions, and he is the only one who manages to do so.
2. The Second Task: The second task is to navigate the underground lake on a magical boat, searching for a beast that is guarding a secret passage. Harry is able to outsmart the beast and retrieve the egg.
3. The Third Task: The third task is to reach the Triwizard Cup before the other champions. Harry must navigate a maze, avoiding various obstacles and creatures, and ultimately outsmart the evil wizard, Barty Crouch Jr.
4. The Dark Forest: During the tournament, Harry and his friends must navigate the Dark Forest, where they encounter various creatures, including the giant spider, Aragog.
5. The Riddle Challenge: During the first 

In [ ]:
prompt = "Name of all Hagrid's pets?"
generated_text = generate_text(prompt, model, tokenizer)
print(generated_text)


[INST] Name of all Hagrid's pets? [/INST]  Hagrid has had several pets over the years, including:

1. Norbert the Norwegian Ridgeback dragon (a large, fire-breathing dragon)
2. Buckbeak the hippogriff (a half-horse, half-eagle creature)
3. Fang, a large, savage-looking boarhound.
4. Griphook, a small, gray-furred goblin.
5. A blind owl named Hedwig.
6. A white owl named Snowy.
7. A large, black cat named Mrs. Norris.
8. A white rat named Scabbers.

All of these pets have been mentioned in the Harry Potter books.


In [ ]:
demo = gr.Interface(
    fn=generate_text,
    inputs=gr.Textbox(label="✨ Enter Your Prompt Here ✨",placeholder="Ask something about Harry Potter...",lines=3,elem_id="input_box"),
    outputs=gr.Textbox(label="🧙‍♂️ Generated Response 🧙‍♀️",lines=10,elem_id="output_box"),
    title="⚡️ Harry Potter LLM Chat ⚡️",
    css="""
        #input_box textarea { background-color: #f0f8ff; border: 2px solid #4b0082; color: #333333;font-size: 16px; padding: 10px;}
        #output_box textarea { background-color: #f5f5f5; border: 2px solid #4b0082; color: #333333; font-size: 16px; padding: 10px;}
        .gr-button { background-color: #4b0082; color: #ffffff; border: none;font-size: 16px; padding: 10px 20px; border-radius: 5px;}
        .gr-button:hover { background-color: #6a0dad;}
    """)

demo.launch()

/usr/local/lib/python3.10/dist-packages/gradio/utils.py:1003: UserWarning: Expected at least 3 arguments for function <function generate_text at 0x7a58f507b5b0>, received 1.
  warnings.warn(


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://ccc8b6025ce0b56dba.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
